In [45]:
import pandas as pd

In [46]:
df = pd.read_csv("../data/parsed_logs_fi.csv")

In [47]:
df["config"].unique()

array(['c10_res44_test_01_bn-relu_base', 'c10_res44_test_02_bn-relu6',
       'c10_res44_test_02_bn-relu6_base', 'c10_res44_test_02_relu6-bn',
       'c10_res44_test_02_relu6-bn_nanfilter',
       'c10_res44_test_02_gelu6_nans', 'c100_res44_test_01_bn-relu_base',
       'c100_res44_test_02_bn-relu6', 'c100_res44_test_02_bn-relu6_base',
       'c100_res44_test_02_relu6-bn',
       'c100_res44_test_02_relu6-bn_nanfilter',
       'c100_res44_test_02_gelu6_nans'], dtype=object)

In [48]:
configs = dict(
    # Baseline C100
    BaseC100="c100_res44_test_01_bn-relu_base.yaml",
    # "c100_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C100
    OrderInvC100="c100_res44_test_02_relu6-bn.yaml",
    # "c100_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C100="c100_res44_test_02_bn-relu6_base.yaml",
    BnRelu6C100="c100_res44_test_02_bn-relu6.yaml",
    # Order inversion + nan filter + Relu6
    OrderINanRelu6C100="c100_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C100
    GeluNanC100="c100_res44_test_02_gelu6_nans.yaml",

    # Baseline C10
    BaseC10="c10_res44_test_01_bn-relu_base.yaml",
    # "c10_res44_test_02_bn-relu6.yaml",
    BnRelu6C10="c10_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C10
    OrderInvC10="c10_res44_test_02_relu6-bn.yaml",
    # "c10_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C10="c10_res44_test_02_bn-relu6_base.yaml",
    OrderINanRelu6C10="c10_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C10
    GeluNanC10="c10_res44_test_02_gelu6_nans.yaml",

    MobileNetC100="cifar100_mobilenetv2_x1_4.yaml",
    MobileNetC10="cifar10_mobilenetv2_x1_4.yaml"
)
configs_inv = {v.replace(".yaml", ""): k for k, v in configs.items()}

In [49]:
configs_df = pd.DataFrame([configs]).T.reset_index().rename(columns={0:"YAML", "index": "config"})

In [50]:
configs_df

,config,YAML
0,BaseC100,c100_res44_test_01_bn-relu_base.yaml
1,OrderInvC100,c100_res44_test_02_relu6-bn.yaml
2,BaseBnRelu6C100,c100_res44_test_02_bn-relu6_base.yaml
3,BnRelu6C100,c100_res44_test_02_bn-relu6.yaml
4,OrderINanRelu6C100,c100_res44_test_02_relu6-bn_nanfilter.yaml
5,GeluNanC100,c100_res44_test_02_gelu6_nans.yaml
6,BaseC10,c10_res44_test_01_bn-relu_base.yaml
7,BnRelu6C10,c10_res44_test_02_bn-relu6.yaml
8,OrderInvC10,c10_res44_test_02_relu6-bn.yaml
9,BaseBnRelu6C10,c10_res44_test_02_bn-relu6_base.yaml


In [51]:
df["config"] = df["config"].apply(lambda x: configs_inv[x])

In [52]:
df["count"] = 1

In [53]:
df[df["sdc"] == 0]

,config,ecc,hostname,fi_model,group,has_end,it,ker_time,acc_time,ker_err,acc_err,sdc,critical_sdc,count
0,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
1,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
4,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
5,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
6,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35982,GeluNanC100,OFF,caroltitaninria,WARP_ZERO_VALUE,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
35984,GeluNanC100,OFF,caroltitaninria,WARP_ZERO_VALUE,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
35990,GeluNanC100,OFF,caroltitaninria,WARP_ZERO_VALUE,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
35995,GeluNanC100,OFF,caroltitaninria,WARP_ZERO_VALUE,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1


In [54]:
to_save = df.groupby(["hostname", "config", "fi_model"]).sum()[["has_end", "sdc", "critical_sdc", "count"]].reset_index()

In [55]:
NEW_NAMES = dict(
        # Baseline C100
    BaseC100="Base",
    # "c100_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C100
    OrderInvC100="OrdInv",
    # "c100_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C100="BaseR6",
    BnRelu6C100="BnR6",
    # Order inversion + nan filter + Relu6
    OrderINanRelu6C100="OrdInvNanR6",
    # Gelu and nan C100
    GeluNanC100="Gelu6Nan",
)
NEW_NAMES.update({k.replace("C100", "C10"): v for k,v in NEW_NAMES.items()})

In [56]:
to_save

,hostname,config,fi_model,has_end,sdc,critical_sdc,count
0,caroltitaninria,BaseBnRelu6C10,FLIP_SINGLE_BIT,400,124.0,1.0,400
1,caroltitaninria,BaseBnRelu6C10,RANDOM_VALUE,400,179.0,2.0,400
2,caroltitaninria,BaseBnRelu6C10,WARP_RANDOM_VALUE,400,246.0,12.0,400
3,caroltitaninria,BaseBnRelu6C10,WARP_ZERO_VALUE,400,251.0,0.0,400
4,caroltitaninria,BaseBnRelu6C10,ZERO_VALUE,400,158.0,0.0,400
...,...,...,...,...,...,...,...
85,caroltitantn,OrderInvC100,FLIP_SINGLE_BIT,400,152.0,0.0,400
86,caroltitantn,OrderInvC100,RANDOM_VALUE,400,224.0,8.0,400
87,caroltitantn,OrderInvC100,WARP_RANDOM_VALUE,400,308.0,37.0,400
88,caroltitantn,OrderInvC100,WARP_ZERO_VALUE,400,293.0,0.0,400


In [57]:
FM_NAMES = dict(FLIP_SINGLE_BIT="Single Bit", ZERO_VALUE="Zero Val",  WARP_ZERO_VALUE="Warp Z.V.", RANDOM_VALUE="Rand Val",  WARP_RANDOM_VALUE="Warp R. V.")

In [58]:
to_save["dataset"] = to_save["config"].apply(lambda x: "Cifar 100" if "C100" in x else "Cifar 10")
to_save["config"] = to_save["config"].apply(lambda x: NEW_NAMES[x])
to_save["fi_model"] = to_save["fi_model"].apply(lambda x: FM_NAMES[x])


In [59]:
to_save

,hostname,config,fi_model,has_end,sdc,critical_sdc,count,dataset
0,caroltitaninria,BaseR6,Single Bit,400,124.0,1.0,400,Cifar 10
1,caroltitaninria,BaseR6,Rand Val,400,179.0,2.0,400,Cifar 10
2,caroltitaninria,BaseR6,Warp R. V.,400,246.0,12.0,400,Cifar 10
3,caroltitaninria,BaseR6,Warp Z.V.,400,251.0,0.0,400,Cifar 10
4,caroltitaninria,BaseR6,Zero Val,400,158.0,0.0,400,Cifar 10
...,...,...,...,...,...,...,...,...
85,caroltitantn,OrdInv,Single Bit,400,152.0,0.0,400,Cifar 100
86,caroltitantn,OrdInv,Rand Val,400,224.0,8.0,400,Cifar 100
87,caroltitantn,OrdInv,Warp R. V.,400,308.0,37.0,400,Cifar 100
88,caroltitantn,OrdInv,Warp Z.V.,400,293.0,0.0,400,Cifar 100


In [67]:
to_save_tail = to_save.groupby(["hostname", "dataset", "config"]).sum().reset_index()

In [68]:
to_save_tail["fi_model"] = "Overall"

In [69]:
to_save_gp = pd.concat([to_save, to_save_tail])

In [70]:
to_save_gp = to_save_gp.groupby(["hostname", "dataset", "fi_model", "config"]).sum()

In [72]:
to_save_gp["avf sdc"] = (to_save_gp["sdc"] - to_save_gp["critical_sdc"])/to_save_gp["count"]
to_save_gp["avf critical sdc"] = to_save_gp["critical_sdc"]/to_save_gp["count"]

In [73]:
to_save_gp

has_end     sdc  critical_sdc  \
hostname        dataset   fi_model config                                       
caroltitaninria Cifar 10  Overall  Base            2000   927.0         213.0   
                                   BaseR6          2000   958.0          15.0   
                                   BnR6            2000   904.0          11.0   
                                   Gelu6Nan        2000  1131.0          15.0   
                                   OrdInv          2000   855.0           7.0   
...                                                 ...     ...           ...   
caroltitantn    Cifar 100 Zero Val BaseR6           400   205.0           0.0   
                                   BnR6             400   183.0           0.0   
                                   Gelu6Nan         400   290.0           0.0   
                                   OrdInv           400   193.0           0.0   
                                   OrdInvNanR6      400   200.0           0.0   

                                                count  avf sdc  \
hostname        dataset   fi_model config                        
caroltitaninria Cifar 10  Overall  Base          2000   0.3570   
                                   BaseR6        2000   0.4715   
                                   BnR6          2000   0.4465   
                                   Gelu6Nan      2000   0.5580   
                                   OrdInv        2000   0.4240   
...                                               ...      ...   
caroltitantn    Cifar 100 Zero Val BaseR6         400   0.5125   
                                   BnR6           400   0.4575   
                                   Gelu6Nan       400   0.7250   
                                   OrdInv         400   0.4825   
                                   OrdInvNanR6    400   0.5000   

                                                avf critical sdc  
hostname        dataset   fi_model config                         
caroltitaninria Cifar 10  Overall  Base                   0.1065  
                                   BaseR6                 0.0075  
                                   BnR6                   0.0055  
                                   Gelu6Nan               0.0075  
                                   OrdInv                 0.0035  
...                                                          ...  
caroltitantn    Cifar 100 Zero Val BaseR6                 0.0000  
                                   BnR6                   0.0000  
                                   Gelu6Nan               0.0000  
                                   OrdInv                 0.0000  
                                   OrdInvNanR6            0.0000  

[108 rows x 6 columns]

In [ ]:
a

In [64]:
to_save_gp.to_excel("/home/fernando/git_research/dsn_2023_diehardnet/data/c10_avf_400faults.xlsx")

In [65]:
to_save_gp

has_end     sdc  critical_sdc  count
dataset   fi_model config                                           
Cifar 10  Overall  Base            2000   927.0         213.0   2000
                   BaseR6          2000   958.0          15.0   2000
                   BnR6            2000   904.0          11.0   2000
                   Gelu6Nan        2000  1131.0          15.0   2000
                   OrdInv          2000   855.0           7.0   2000
...                                 ...     ...           ...    ...
Cifar 100 Zero Val BaseR6           800   354.0           0.0    800
                   BnR6             800   306.0           0.0    800
                   Gelu6Nan         800   538.0           0.0    800
                   OrdInv           800   341.0           0.0    800
                   OrdInvNanR6      800   373.0           0.0    800

[72 rows x 4 columns]

In [66]:
df["hostname"].unique()

array(['caroltitaninria', 'caroltitantn'], dtype=object)